## Create Monte Carlo method to integrate posterior

In [1]:
%pylab inline
import mcint
import numpy.random
from Chempy.parameter import ModelParameters
from Chempy.cem_function import posterior_function_for_integration
a = ModelParameters()

Populating the interactive namespace from numpy and matplotlib


### TEST with different random generators e.g. gaussian

In [3]:
# Define function to return posterior predictions (must be exponentiated)
def posterior(theta):
    a = ModelParameters()
    post,_ = posterior_function_for_integration(np.array(theta))
    posterior = np.exp(post)
    return posterior

sigma = [] # Read prior sigma from parameter file
for i,param_name in enumerate(a.to_optimize):
    sigma.append(a.priors.get(param_name)[1])
    
Nsigma = 1 # Gives width in sigma space
Nsamples=100 # Number of MC samples

def sampler():
    while True:
        yield random.uniform(np.array(a.p0)-Nsigma*np.array(sigma),np.array(a.p0)+Nsigma*np.array(sigma),size=6)

# Calculate total domain size used
domain_size = np.prod(sigma)*(2*Nsigma)**len(sigma)

In [ ]:
%prun result, error = mcint.integrate(posterior, sampler(),n=Nsamples,measure=domain_size)  

In [ ]:
print(result,error)

In [4]:
%timeit -r 1 -n 1 posterior([-3,-3,-0.8,-0.5,0.6,0.5])

1 loop, best of 1: 318 ms per loop


In [ ]:
d1 = []
d2 = []
d3 = []
for i in range(500):
    param = random.uniform(np.array(a.p0)-Nsigma*np.array(sigma),np.array(a.p0)+Nsigma*np.array(sigma),size=6)
    param[2] = a.p0[2]
    param[3] = a.p0[3]
    param[4] = a.p0[4]
    param[5] = a.p0[5]
    post,_ = posterior_function_many_stars(param,a)
    posterior = np.exp(post)
    d1.append(param[0])
    d2.append(param[1])
    d3.append(posterior)


In [ ]:
plt.scatter(d1,d2,c=np.log10(d3))
plt.colorbar()

In [ ]:
plt.scatter(d1,d3)


## New posterior function with error marginalization

In [ ]:
from Chempy.cem_function import posterior_function_many_stars

def posterior(theta):
    a = ModelParameters()
    post,_ = posterior_function_many_stars(np.array(param),np.inf,a.elements_to_trace)
    posterior = np.exp(post)
    return posterior

In [ ]:
a = ModelParameters()
param = [-3,-3,-0.8,-0.3,0.5,0.6]
pos,abun = posterior_function_many_stars(np.array(param),np.inf,a.elements_to_trace)

In [ ]:
pos,abun = posterior_function_many_stars(np.array(param),np.inf,a.elements_to_trace)